Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import os

# Loading the dataset
FILE_NAME = "telco.csv"
RAW_DATA_PATH = os.path.join("..", "data", "raw", FILE_NAME)
df = pd.read_csv(RAW_DATA_PATH)

Removing columns related to the target

In [ ]:
# Remove columns that are directly related to the target variable 'Churn Label' to avoid data leakage
cols_to_drop = ['Churn Reason', 'Churn Score', 'Churn Category']
df_clean = df.drop(columns=cols_to_drop, errors='ignore')

print(f"🗑️ Removed columns: {cols_to_drop}")
print(f"✅ Dataset shape after dropping columns: {df_clean.shape}")

🗑️ Removed columns: ['Churn Reason', 'Churn Score', 'Churn Category']
✅ Dataset shape after dropping columns: (7043, 47)


Detect NaN Values

In [4]:

print("\n--- Check for missing values ---")
# Count NaN values in each column
nan_counts = df_clean.isnull().sum()
print(nan_counts[nan_counts > 0])

# Look for empty strings in 'Total Charges' which should be numeric
# Pandas considers blank strings as non-NaN, so we need to check for them explicitly
empty_strings = (df_clean['Total Charges'] == ' ').sum()

print(f"\nEmpty strings in 'Total Charges': {empty_strings}")
# Try converting 'Total Charges' to numeric, coercing errors to NaN, to find hidden non-numeric values
temp_charges = pd.to_numeric(df_clean['Total Charges'], errors='coerce')
df_clean['Total Charges'] = temp_charges
hidden_nans = temp_charges.isnull().sum()
print(f"Hidden non-numeric values in 'Total Charges': {hidden_nans}")



--- Check for missing values ---
Offer            3877
Internet Type    1526
dtype: int64

Empty strings in 'Total Charges': 0
Hidden non-numeric values in 'Total Charges': 0


Imputation

In [10]:
# Handling missing values in 'Total Charges'
# Strategy: Since 'Total Charges' is likely a cumulative amount, it may make sense to fill NaN values with 0, 
# assuming that these customers haven't been charged yet.
df_clean['Total Charges'] = df_clean['Total Charges'].fillna(0)
print("✅ Missing values in 'Total Charges' filled with 0.")

✅ Missing values in 'Total Charges' filled with 0.


Encode Target

In [11]:
# --- TARGET ENCODING ---
# Encode the target variable 'Churn Label' into a binary numeric format (1 for 'Yes', 0 for 'No')
target_col = 'Churn Label' 

if target_col in df_clean.columns:
    print(f"\nEncoding Target '{target_col}'...")
    df_clean['Churn'] = df_clean[target_col].map({'Yes': 1, 'No': 0})
    df_clean = df_clean.drop(columns=[target_col])
elif 'Churn Value' in df_clean.columns:
    print("\nTarget 'Churn Value' already exists and is numeric. Renaming to 'Churn'.")
    df_clean.rename(columns={'Churn Value': 'Churn'}, inplace=True)

# Check the distribution of the target variable after encoding
print("\nTarget variable distribution (after encoding):")
print(df_clean['Churn'].value_counts(normalize=True))




Encoding Target 'Churn Label'...

Target variable distribution (after encoding):
Churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64


Saving processed data

In [12]:
# --- SAVE PROCESSED DATA ---
PROCESSED_PATH = os.path.join("..", "data", "processed", "telco_clean.csv")
os.makedirs(os.path.dirname(PROCESSED_PATH), exist_ok=True)

df_clean.to_csv(PROCESSED_PATH, index=False)
print(f"\n💾 Processed file saved to: {PROCESSED_PATH}")
print(f"Final dimensions: {df_clean.shape}")


💾 Processed file saved to: ..\data\processed\telco_clean.csv
Final dimensions: (7043, 47)
